In [9]:
import xarray as xr
import numpy as np

ds = xr.open_dataset("/Users/rem76/Desktop/Climate_change_health/Data/Temperature_data/Historical/Hourly/7ba7ebaaf13bddcb13b9d30aaa6f1512.nc")  # contains t2m (K) and d2m (K)
ds_weekly = ds.resample(valid_time="7D").mean()
# 
# 
# 
# Tc  = ds["t2m"] - 273.15
# Tdc = ds["d2m"] - 273.15
# 
# es = 6.112 * np.exp(17.625 * Tc  / (243.04 + Tc))   # hPa
# e  = 6.112 * np.exp(17.625 * Tdc / (243.04 + Tdc))  # hPa
# 
# rh = 100.0 * (e / es)
# rh = rh.clip(0, 100)
# 
# rh = rh.rename("rh2m")
# rh.attrs.update(long_name="2 m relative humidity", units="%")


In [10]:
ds_weekly

<xarray.Dataset> Size: 1GB
Dimensions:     (valid_time: 22, latitude: 1801, longitude: 3600)
Coordinates:
    number      int64 8B 0
  * latitude    (latitude) float64 14kB 90.0 89.9 89.8 ... -89.8 -89.9 -90.0
  * longitude   (longitude) float64 29kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
  * valid_time  (valid_time) datetime64[ns] 176B 2019-01-01 ... 2019-05-28
Data variables:
    d2m         (valid_time, latitude, longitude) float32 571MB nan ... 215.5
    t2m         (valid_time, latitude, longitude) float32 571MB nan ... 219.4
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-20T09:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [12]:
import pandas as pd

# ----------------
# 2019 data
# ----------------
mod_b_2019 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_B.csv")
mod_e_2019 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_E.csv")
geo_2019   = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/householdgeovariables_ihs5.csv")

merged_2019 = pd.merge(mod_b_2019, mod_e_2019, on=["case_id", "HHID", "PID"], how="inner")
merged_2019 = pd.merge(merged_2019, geo_2019, on=["case_id"], how="left")
merged_2019["survey_year"] = 2019

# ----------------
# 2016 data
# ----------------
mod_b_2016 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2016_IHS-IV_v04_M_CSV/household/HH_MOD_B.csv")
mod_e_2016 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2016_IHS-IV_v04_M_CSV/household/HH_MOD_E.csv")
geo_2016   = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2016_IHS-IV_v04_M_CSV/household/household_geovariables/householdgeovariables_ihs4.csv")

merged_2016 = pd.merge(mod_b_2016, mod_e_2016, on=["case_id", "HHID", "PID"], how="inner")
merged_2016 = pd.merge(merged_2016, geo_2016, on=["case_id"], how="left")
merged_2016["survey_year"] = 2016

# ----------------
# Combine 2016 + 2019
# ----------------
combined = pd.concat([merged_2016, merged_2019], ignore_index=True, sort=False)

# ----------------
# Filtering logic (same as before)
# ----------------
keywords = ["HEALTH WORKER", "DOCTOR", "NURSE"]
mask_keywords = combined.apply(
    lambda row: row.astype(str).str.contains('|'.join(keywords), case=False, na=False).any(), axis=1
)
subset_health = combined[mask_keywords]

subset_health["hh_e11"] = pd.to_numeric(subset_health["hh_e11"], errors="coerce")
sum_hours_health = subset_health["hh_e11"].sum()

subset_weather = subset_health[subset_health["hh_e15"].str.contains("BAD WEATHER|NATURAL DISASTER", case=False, na=False)]

print("Sum of hh_e11 for HEALTH WORKER/DOCTOR/NURSE subset:", sum_hours_health)
print("Subset with BAD WEATHER or NATURAL DISASTER in hh_e15:")
#print(subset_weather.head())


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_73317/2884019967.py:6: DtypeWarning: Columns (17,25,35,37,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  mod_b_2019 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_B.csv")
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_73317/2884019967.py:7: DtypeWarning: Columns (38,42,44,45,46,64,88,90,92,93,94,95,105,117,122,125,126,133,158) have mixed types. Specify dtype option on import or set low_memory=False.
  mod_e_2019 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_E.csv")
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_73317/2884019967.py:17: DtypeWarning: Columns (17,25) have mixed types. Specify dtype option on import or set low_memory=False.
  mod_b_2016 = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_H

Sum of hh_e11 for HEALTH WORKER/DOCTOR/NURSE subset: 3670.0
Subset with BAD WEATHER or NATURAL DISASTER in hh_e15:


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_73317/2884019967.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health["hh_e11"] = pd.to_numeric(subset_health["hh_e11"], errors="coerce")


In [8]:
subset_health

,case_id,HHID_x,PID,hh_b03,hh_b04,hh_b04a,hh_b05_2,hh_b05_2_1,hh_b05a,hh_b05b,...,h2019_wetQstart,h2019_wetQ,h2018_ndvi_avg,h2018_ndvi_max,h2019_ndvi_avg,h2019_ndvi_max,ndvi_avg,ndvi_max,ea_lat_mod,ea_lon_mod
1623,102011240108,9f46b5498b00459f80c79bc67917a72a,2,2,2,2.0,2.0,NaN,33.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1958,102011290374,fd32ba7311914e9fa444ef644968edf9,1,2,1,1.0,2.0,NaN,55.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3213,102201470596,e64e6a8599414aa898a125aba54d6376,1,1,1,1.0,2.0,NaN,41.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6500,104071880309,8673ac634ced41458f7c05da121ad6a3,1,1,1,1.0,2.0,NaN,49.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6794,104201930020,abb9daeef3b74c7d91cfc2cd9322c14b,1,2,1,1.0,1.0,2.0,52.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102978,315317950130,NaN,1,MALE,HEAD,YES,NaN,NaN,79.0,NaN,...,17.0,736.0,0.154,0.308,0.110,0.357,0.173,0.352,-15.72683,35.018299
103077,315327970003,NaN,2,FEMALE,WIFE/HUSBAND,YES,NaN,NaN,36.0,NaN,...,17.0,737.0,0.162,0.326,0.117,0.402,0.173,0.365,-15.72466,35.070820
103279,315337990150,NaN,1,MALE,HEAD,NO,NaN,NaN,44.0,NaN,...,17.0,738.0,0.169,0.345,0.117,0.408,0.175,0.371,-15.73670,35.076569
103732,315408060105,NaN,2,FEMALE,WIFE/HUSBAND,YES,NaN,NaN,30.0,NaN,...,17.0,738.0,0.157,0.353,0.116,0.386,0.173,0.357,-15.75323,34.971771


In [13]:
subset_weather

,case_id,HHID_x,PID,hh_b03,hh_b04,hh_b04a,hh_b05_2,hh_b05_2_1,hh_b05a,hh_b05b,...,h2019_wetQstart,h2019_wetQ,h2018_ndvi_avg,h2018_ndvi_max,h2019_ndvi_avg,h2019_ndvi_max,ndvi_avg,ndvi_max,ea_lat_mod,ea_lon_mod


In [24]:

keywords = ["HEALTH WORKER", "DOCTOR", "NURSE"]
pattern = "|".join(keywords)  # "HEALTH WORKER|DOCTOR|NURSE"

subset = mod_e[mod_e["hh_e19a"].str.contains(pattern, case=False, na=False)]
subset

,case_id,HHID,PID,hh_e03,hh_e04,hh_e05,hh_e06,hh_e06_1a,hh_e06_1b,hh_e06_1c,...,hh_e63,hh_e64,hh_e65,hh_e66,hh_e67,hh_e68,hh_e69,hh_e70,hh_e71,hh_e71_oth
2765,102041370093,7fbd200d6dc54ddea90a6597c7686557,1,NO,2.0,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
3365,102201450188,76bbaf58d54f4c91b63512d6b77d0bfc,2,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
6108,104071820041,696daf3ac40043a0aa20f5a416fbbc9b,1,YES,NaN,0.2,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,Yes,Yes,3.0,No,NaN,NaN
8523,107362180099,d516c1cfb2ce4c838259ba60226e511a,2,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
8547,107362180149,ab5e728f068e42359a04448f2231c901,1,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
8720,107372210076,0e1fdf7b916546f7994528971a7ed1fa,2,YES,NaN,0.0,0.0,YES,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
8983,107392250204,cfe1060884514a03a665e69d32d12dea,3,NO,1.0,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,Yes,Yes,Yes,4.0,Yes,To have a better paid job,NaN
9067,107392260141,3593b5cc0f0a42f4bdec98b89a0ed244,1,YES,NaN,0.0,0.0,YES,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
9074,107392260165,e3200be4c72042fe85d5189d7d6c954c,2,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,NaN,NaN,NaN
9294,107402300015,df216a6d39cb42f4a996bc526f55b381,1,NO,2.0,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
